# 1) ¿Qué “cuenta” el término ∂(dI/dt)/∂S?

Piensa en el **Jacobiano** como una foto de “qué tan sensible” es el sistema en un punto.  
Ese término, en palabras sencillas, mide **cuánto cambia la velocidad de crecimiento de los casos si aumentan los susceptibles**.  

- Si es **positivo y grande**, significa que más personas susceptibles hacen que los casos crezcan más rápido.  
- Si es **pequeño o cercano a cero**, tener más susceptibles ya no acelera tanto el crecimiento de la infección.  

En el modelo **SIR básico con dinámica vital**, este término es **proporcional al número de infectados**, reflejando que los susceptibles y el contagio están acoplados:  
cuantos más susceptibles haya, más “pista” tiene el virus para avanzar.  


---

# 2) ¿Por qué el equilibrio libre de enfermedad es “inestable” cuando R₀ > 1?

“Inestable” significa que **una pequeña chispa de casos no se apaga sola**:  
si aparece un pequeño grupo de infectados, el sistema **se aleja del punto sin enfermedad** y la infección **despega**.  

Por eso, una política de “esperar y ver” es riesgosa: no actuar temprano permite que esa chispa crezca y se convierta en un brote sostenido.  


---

# 3) ¿Qué quiere decir que el equilibrio endémico “suele ser estable”?

“Estable” significa que, aunque haya sacudidas (por ejemplo, aumentos repentinos de casos), el sistema **tiende a regresar a un nivel constante** de susceptibles e infectados.  

Si surge una variante más contagiosa que dispara los casos temporalmente, la **estabilidad del equilibrio endémico** implica que, tras la ola, el sistema volverá hacia su nivel original (o hacia un **nuevo equilibrio endémico** si cambian los parámetros de transmisión).  

---

# 4) ¿Cómo se ve un “equilibrio estable” en un modelo basado en agentes (MBA) con azar?

En un **modelo basado en agentes (MBA)** el sistema nunca se “clava” exactamente en un punto.  
En cambio, lo que se observa es que, tras la fase transitoria, la trayectoria **oscila alrededor de una zona del plano (S, I)** sin alejarse mucho.  

En otras palabras, aparece un **“nubeo” de puntos** alrededor del nivel endémico: hay fluctuaciones permanentes por el azar,  
pero el sistema **se mantiene cerca del equilibrio en promedio**.  

---

## Referencias breves

- **Hethcote, H.W.** (2000). *The Mathematics of Infectious Diseases*. *SIAM Review*: umbral **R₀** y estabilidad/inestabilidad del equilibrio libre de enfermedad.  
  [epubs.siam.org](https://epubs.siam.org)

- **Keeling, M.J., Rohani, P.** *Modeling Infectious Diseases in Humans and Animals*, cap. 2: equilibrio endémico y sensibilidad del sistema.  
  [math.uchicago.edu](https://math.uchicago.edu)

- **van den Driessche, P., Watmough, J.** (2017, revisión): método de **R₀** y estabilidad de equilibrios.  
  [PMC](https://www.ncbi.nlm.nih.gov/pmc/)

- **Allen, L.J.S.** (2017): en modelos estocásticos, las trayectorias fluctúan alrededor del equilibrio endémico del modelo determinista.  
  [PMC](https://www.ncbi.nlm.nih.gov/pmc/)

- **Greenwood, P.E., Gordillo, L.F.** Relación entre modelos estocásticos y su “esqueleto” determinista (fluctuaciones alrededor del equilibrio).  
  [aimath.org](https://aimath.org)


## **Práctica Parte 2**

In [1]:
import sympy as sp
import numpy as np

# 1) VARIABLES Y PARAMETROS SIMBOLICOS
S, I = sp.symbols('S I', real=True)
beta, gamma, mu, N = sp.symbols('beta gamma mu N', positive=True, real=True)

# 1b) ECUACIONES DIFERENCIALES (expresiones sympy)
dS = mu*N - (beta*S*I)/N - mu*S + gamma*I
dI = (beta*S*I)/N - (gamma+mu)*I

# 1c) MATRICES PARA JACOBIANO
F = sp.Matrix([dS, dI])     # matriz de ecuaciones
X = sp.Matrix([S, I])       # matriz de variables

# 1d) JACOBIANO SIMBOLICO
J = F.jacobian(X)

print("=== (1) Jacobiano simbólico J(S,I) ===")
sp.pprint(J)
print()

# 2) AUTOVALORES SIMBOLICOS EN ELE (S=N, I=0)
J_ELE = J.subs({S: N, I: 0})
eig_ELE = J_ELE.eigenvals()  # dict {eigenvalue: multiplicity}

print("=== (2) Autovalores simbólicos en ELE (S=N, I=0) ===")
for lam, mult in eig_ELE.items():
    print(f"λ (mult={mult}) =", sp.simplify(lam))
print()

# ============================================
# 3) PARÁMETROS NUMÉRICOS (¡RELLENAR CON TUS VALORES!)
# Ejemplo de referencia (puedes sobreescribir):
# N_val     = 1_000_000
# beta_val  = 0.6
# gamma_val = 0.2
# mu_val    = 0.0001
# ============================================
N_val     = 1_000_000   # <-- COPIA TUS VALORES DE LA PARTE 1
beta_val  = 0.6         # <--
gamma_val = 0.2         # <--
mu_val    = 0.0001      # <--

param_num = {
    N: N_val,
    beta: beta_val,
    gamma: gamma_val,
    mu: mu_val
}

# 3a) CALCULO NUMERICO DEL EQUILIBRIO ENDÉMICO (si existe)
# S* = N*(gamma+mu)/beta, I* = N - S*
R0 = beta_val / (gamma_val + mu_val)
S_star = N_val * (gamma_val + mu_val) / beta_val
I_star = N_val - S_star

print("=== (3) Parámetros numéricos y equilibrio endémico ===")
print(f"N = {N_val}, beta = {beta_val}, gamma = {gamma_val}, mu = {mu_val}")
print(f"R0 = beta/(gamma+mu) = {R0:.6f}")
if R0 <= 1:
    print("⚠️  R0 ≤ 1 → No existe equilibrio endémico positivo (I* ≤ 0).")
else:
    print("R0 > 1 → Existe equilibrio endémico.")
print(f"S* = {S_star:.6f}")
print(f"I* = {I_star:.6f}")
print()

# 3b) JACOBIANO NUMERICO EN EL ENDÉMICO
J_endemic_sym = J.subs(param_num).subs({S: S_star, I: I_star})  # aún objeto sympy, pero numérico
print("=== (3c) Jacobiano numérico en el equilibrio endémico ===")
sp.pprint(sp.N(J_endemic_sym, 10))
print()

# 4) AUTOVALORES NUMERICOS (NumPy)
J_endemic_np = np.array(J_endemic_sym, dtype=np.float64)  # convertir a array de numpy
eigvals_endemic = np.linalg.eigvals(J_endemic_np)

print("=== (4) Autovalores numéricos del Jacobiano en el equilibrio endémico ===")
# Pueden ser complejos
for idx, lam in enumerate(eigvals_endemic, 1):
    if abs(lam.imag) < 1e-12:
        print(f"λ{idx} = {lam.real:.10f}")
    else:
        print(f"λ{idx} = {lam.real:.10f} {'+' if lam.imag>=0 else '-'} {abs(lam.imag):.10f} i")

# 5) RESUMEN CLARO
print("\n=== (5) Resumen solicitado ===")
print("- Jacobiano simbólico mostrado en (1).")
print("- Autovalores simbólicos en ELE mostrados en (2).")
print("- Jacobiano numérico en endémico mostrado en (3c).")
print("- Autovalores numéricos en endémico mostrados en (4).")

# (Opcional) Clasificación de estabilidad simple
def estabilidad_desde_autovalores(vals, tol=1e-12):
    re = [v.real for v in vals]
    if max(re) < -tol:
        return "Estable (atractor): partes reales negativas."
    if min(re) > tol:
        return "Inestable (repulsor): partes reales positivas."
    return "Silla o caso límite: signos mixtos o cercanos a 0."

print("\n=== (Opcional) Diagnóstico de estabilidad (numérico, endémico) ===")
print(estabilidad_desde_autovalores(eigvals_endemic))

=== (1) Jacobiano simbólico J(S,I) ===
⎡  I⋅β            S⋅β   ⎤
⎢- ─── - μ    γ - ───   ⎥
⎢   N              N    ⎥
⎢                       ⎥
⎢   I⋅β              S⋅β⎥
⎢   ───     -γ - μ + ───⎥
⎣    N                N ⎦

=== (2) Autovalores simbólicos en ELE (S=N, I=0) ===
λ (mult=1) = beta - gamma - mu
λ (mult=1) = -mu

=== (3) Parámetros numéricos y equilibrio endémico ===
N = 1000000, beta = 0.6, gamma = 0.2, mu = 0.0001
R0 = beta/(gamma+mu) = 2.998501
R0 > 1 → Existe equilibrio endémico.
S* = 333500.000000
I* = 666500.000000

=== (3c) Jacobiano numérico en el equilibrio endémico ===
⎡ -0.4   -0.0001⎤
⎢               ⎥
⎣0.3999     0   ⎦

=== (4) Autovalores numéricos del Jacobiano en el equilibrio endémico ===
λ1 = -0.3999000000
λ2 = -0.0001000000

=== (5) Resumen solicitado ===
- Jacobiano simbólico mostrado en (1).
- Autovalores simbólicos en ELE mostrados en (2).
- Jacobiano numérico en endémico mostrado en (3c).
- Autovalores numéricos en endémico mostrados en (4).

=== (Opcion